In [1]:
import numpy as np
import pandas as pd

In [2]:
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

-------

#

In [3]:
outdir = '/Users/ryan/Dropbox/Projects/Research/tikhonov/barcode-bias/barcode-bias/chen-case-study/data/chen-2023-preprocessed-data/'

-------

# Process Haploid Bulk Fitness Assay (hBFA) data

In [4]:
rawDataDir_hBFA1 = '/Users/ryan/Dropbox/Projects/Research/tikhonov/barcode-bias/barcode-bias/chen-case-study/data/chen-2023-elife-data/'

#### Load & organize raw data

##### Load the raw Chen et al. (2023) barcode counts data table

(rows: variants; cols: barcode counts for each experimental treatment/rep/timept and variant metadata)

In [5]:
rawCountsTable_hBFA1 = pd.read_csv(rawDataDir_hBFA1+'hBFA1_counts_with_env_info.csv')
# rawCountsTable_hBFA1

#### Extract the counts data (excluding metadata) from the variant and ancestor datasets

In [6]:
metadata_columns = ['Full.BC', 'Diverse.BC', 'Environment.BC', 'Total.Counts', 'Subpool.Environment', 'Which.Subpools', 'Putative.Environment']

In [7]:
rawCounts_hBFA1 = rawCountsTable_hBFA1.drop(columns=metadata_columns)
rawCounts_hBFA1

hBFA1-21C-R1-Time8  hBFA1-21C-R1-Time16  hBFA1-21C-R1-Time24  \
0                  18075                34976                73925   
1                 109316               117037               102934   
2                   6539                 4508                 4751   
3                  31099                42058                42356   
4                   4250                 7508                13340   
...                  ...                  ...                  ...   
2581                   1                    2                    4   
2582                   3                    3                    5   
2583                   3                    5                    4   
2584                   5                    4                    4   
2585                   4                    4                    3   

      hBFA1-21C-R1-Time40  hBFA1-21C-R2-Time8  hBFA1-21C-R2-Time16  \
0                  379540               19395                33224   
1                   81035              122720               113384   
2                    3545                7334                 4549   
3                   38478               39228                41195   
4                   61474                4878                 7708   
...                   ...                 ...                  ...   
2581                    2                   5                    6   
2582                    1                   7                    5   
2583                    2                   3                    7   
2584                    2                   8                    6   
2585                    2                   2                    3   

      hBFA1-21C-R2-Time24  hBFA1-21C-R2-Time40  hBFA1-37C-R1-Time8  \
0                   67414               473720               36413   
1                   95658                83279              140343   
2                    4273                 3511                7771   
3                   42935                51296               32634   
4                   13778                94633                6576   
...                   ...                  ...                 ...   
2581                    7                    9                   7   
2582                    5                    5                   4   
2583                    6                    3                   1   
2584                    2                   13                   0   
2585                    2                    6                   0   

      hBFA1-37C-R1-Time16  hBFA1-37C-R1-Time24  hBFA1-37C-R1-Time40  \
0                   72215               120048               323260   
1                  131053               103226                65894   
2                    6269                 9488                11685   
3                   23369                18619                 7488   
4                   14705                29220                76955   
...                   ...                  ...                  ...   
2581                    2                    3                    2   
2582                    0                    4                    3   
2583                    0                    3                    3   
2584                    0                    4                    0   
2585                    0                    5                    1   

      hBFA1-37C-R2-Time8  hBFA1-37C-R2-Time16  hBFA1-37C-R2-Time24  \
0                  35264                76376               128764   
1                 161542               152263               127606   
2                  12360                 7577                 5588   
3                  40855                28633                17674   
4                   7438                15378                20278   
...                  ...                  ...                  ...   
2581                   4                    7                    3   
2582                   2                    3                    5   


#### Extract the metadata for variants from the variants dataset

In [8]:
variantsInfo_hBFA1 = rawCountsTable_hBFA1[metadata_columns].copy()  # this .copy() prevents variantsInfo_hBFA1 from being seen as a slice internally
variantsInfo_hBFA1 = variantsInfo_hBFA1.rename(columns={'Full.BC':'barcode_full', 'Diverse.BC':'barcode_diverse', 'Environment.BC':'barcode_environment', 'Total.Counts':'total_counts',
                                            'Subpool.Environment':'subpool_environment', 'Which.Subpools':'which_subpools', 'Putative.Environment':'putative_environment'})
variantsInfo_hBFA1

barcode_full  \
0     ATAAAAAAGCACAAGCCTTTTGACGACACCTAAATTAGTTATCCAT...   
1     CGATAAACCCACAATATGATTATCGGTACACAAGAGGGTTTCTGTT...   
2     CGCACAAGAAAGAATAATCTTGAATTGGTAAAACCAGATTTGGCAT...   
3     ACAAAAAGATATAACAAGCTTGAAGACCCGAAAAAGTTTTTTATCT...   
4     AATAAAAGAAGGAAAAGCATTTAAACAAACAAACTTTCTTTTTTCT...   
...                                                 ...   
2581  CACGCAAAACTTTATTCAATAACTTCGTCCGCCAATCATATTGTAC...   
2582  CTATGAAAAGACAACAAGATTGCCTACTATTAACTGATTTTTCTCT...   
2583  CGTTAGAATTCTAAACCTGGTTCAACTATTGGAACATCTTTAGTAT...   
2584  CCGTCAATCATTAACTTAGTTTCTTACCAGAAATCATATTTTTATT...   
2585  TTATTAATATAGAAAGCAATTTTAATAGAAAAACCATTTTCGTATT...   

                   barcode_diverse         barcode_environment  total_counts  \
0       ATAAAAAAGCACAAGCCTTTTGACGA  CACCTAAATTAGTTATCCATTCGGCT       9616313   
1       CGATAAACCCACAATATGATTATCGG  TACACAAGAGGGTTTCTGTTTTATGG       6648391   
2       CGCACAAGAAAGAATAATCTTGAATT  GGTAAAACCAGATTTGGCATTCACTA       3368497   
3       ACAAAAAGATATAACAAGCTTGAAGA  CCCGAAAAAGTTTTTTATCTTCAATG       2082320   
4       AATAAAAGAAGGAAAAGCATTTAAAC  AAACAAACTTTCTTTTTTCTTTATGC       2013526   
...                            ...                         ...           ...   
2581  CACGCAAAACTTTATTCAATAACTTCGT  CCGCCAATCATATTGTACTTTCAAGG           267   
2582    CTATGAAAAGACAACAAGATTGCCTA  CTATTAACTGATTTTTCTCTTTCGAA           243   
2583   CGTTAGAATTCTAAACCTGGTTCAACT  ATTGGAACATCTTTAGTATTTAATAC           214   
2584    CCGTCAATCATTAACTTAGTTTCTTA  CCAGAAATCATATTTTTATTTATTAA           139   
2585    TTATTAATATAGAAAGCAATTTTAAT  AGAAAAACCATTTTCGTATTTTATAG            96   

     subpool_environment                which_subpools putative_environment  
0               not_read                      not_read                SC_2N  
1               not_read                      not_read             SC_alpha  
2          GlyEtOH_alpha  -R1-Subpool.-R1-autodiploids        GlyEtOH_alpha  
3             FLC4_alpha                   -R1-Subpool           FLC4_alpha  
4              YPD_alpha                   -R1-Subpool      contam-pH3_8_2N  
...                  ...                           ...                  ...  
2581            not_read                      not_read             SC_alpha  
2582         pH7_3_alpha                   -R2-Subpool   sanger-pH7_3_alpha  
2583           37C_alpha                   -R1-Subpool              unknown  
2584           21C_alpha                   -R1-Subpool            21C_alpha  
2585           YPD_alpha                   -R1-Subpool            YPD_alpha  

[2586 rows x 7 columns]

#### Parse treatment, rep, and timepoint information from sample labels (column names)

In [9]:
sample_columns_raw = rawCounts_hBFA1.columns
sample_columns_raw.values

array(['hBFA1-21C-R1-Time8', 'hBFA1-21C-R1-Time16', 'hBFA1-21C-R1-Time24',
       'hBFA1-21C-R1-Time40', 'hBFA1-21C-R2-Time8', 'hBFA1-21C-R2-Time16',
       'hBFA1-21C-R2-Time24', 'hBFA1-21C-R2-Time40', 'hBFA1-37C-R1-Time8',
       'hBFA1-37C-R1-Time16', 'hBFA1-37C-R1-Time24',
       'hBFA1-37C-R1-Time40', 'hBFA1-37C-R2-Time8', 'hBFA1-37C-R2-Time16',
       'hBFA1-37C-R2-Time24', 'hBFA1-37C-R2-Time40',
       'hBFA1-48Hr-R1-Time8', 'hBFA1-48Hr-R1-Time16',
       'hBFA1-48Hr-R1-Time24', 'hBFA1-48Hr-R1-Time40',
       'hBFA1-48Hr-R2-Time8', 'hBFA1-48Hr-R2-Time16',
       'hBFA1-48Hr-R2-Time24', 'hBFA1-48Hr-R2-Time40',
       'hBFA1-FLC4-R2-Time8', 'hBFA1-FLC4-R2-Time16',
       'hBFA1-FLC4-R2-Time24', 'hBFA1-FLC4-R2-Time40',
       'hBFA1-GlyEtOH-R1-Time8', 'hBFA1-GlyEtOH-R1-Time16',
       'hBFA1-GlyEtOH-R1-Time24', 'hBFA1-GlyEtOH-R1-Time40',
       'hBFA1-GlyEtOH-R2-Time8', 'hBFA1-GlyEtOH-R2-Time16',
       'hBFA1-GlyEtOH-R2-Time24', 'hBFA1-GlyEtOH-R2-Time40',
       'hBFA1-SC-R1-Time8

In [10]:
samplesInfo_hBFA1 = []

sample_renames = {}

for sample in rawCounts_hBFA1.columns:
    sample_info = {}
    sample_info['chen_sample_label'] = sample
    sample_label_parts = sample.split('-')
    sample_info['assay_set'] = sample_label_parts[0]
    sample_info['treatment'] = '.'.join(sample_label_parts[1].split('_'))
    sample_info['rep']       = sample_label_parts[2]
    sample_info['timept']    = sample_label_parts[3][4:]
    sample_info['assay']     = sample_info['assay_set'] + ':' + sample_info['treatment'] + '-' + sample_info['rep']
    sample_info['sample']    = sample_info['assay'] + '-' + sample_info['timept']
    sample_info['total_raw_count'] = rawCounts_hBFA1[sample].sum()
    sample_renames[sample] = sample_info['sample']
    samplesInfo_hBFA1.append(sample_info)
    
samplesInfo_hBFA1 = pd.DataFrame(samplesInfo_hBFA1)

rawCounts_hBFA1 = rawCounts_hBFA1.rename(columns=sample_renames)

In [11]:
samplesInfo_hBFA1['timept'] = pd.to_numeric(samplesInfo_hBFA1['timept'])

In [12]:
# Sort dataframe for convenience:
samplesInfo_hBFA1.sort_values(by=['assay_set', 'treatment', 'rep', 'timept'], ascending=[True, True, True, True], ignore_index=True, inplace=True)

In [13]:
# Reorder columns for cosmetic reasons
samplesInfo_hBFA1 = samplesInfo_hBFA1[['chen_sample_label', 'sample', 'assay', 'assay_set', 'treatment', 'rep', 'timept', 'total_raw_count']]

In [14]:
samplesInfo_hBFA1

chen_sample_label             sample           assay assay_set  \
0      hBFA1-21C-R1-Time8     hBFA1:21C-R1-8    hBFA1:21C-R1     hBFA1   
1     hBFA1-21C-R1-Time16    hBFA1:21C-R1-16    hBFA1:21C-R1     hBFA1   
2     hBFA1-21C-R1-Time24    hBFA1:21C-R1-24    hBFA1:21C-R1     hBFA1   
3     hBFA1-21C-R1-Time40    hBFA1:21C-R1-40    hBFA1:21C-R1     hBFA1   
4      hBFA1-21C-R2-Time8     hBFA1:21C-R2-8    hBFA1:21C-R2     hBFA1   
..                    ...                ...             ...       ...   
63  hBFA1-pH7_3-R1-Time40  hBFA1:pH7.3-R1-40  hBFA1:pH7.3-R1     hBFA1   
64   hBFA1-pH7_3-R2-Time8   hBFA1:pH7.3-R2-8  hBFA1:pH7.3-R2     hBFA1   
65  hBFA1-pH7_3-R2-Time16  hBFA1:pH7.3-R2-16  hBFA1:pH7.3-R2     hBFA1   
66  hBFA1-pH7_3-R2-Time24  hBFA1:pH7.3-R2-24  hBFA1:pH7.3-R2     hBFA1   
67  hBFA1-pH7_3-R2-Time40  hBFA1:pH7.3-R2-40  hBFA1:pH7.3-R2     hBFA1   

   treatment rep  timept  total_raw_count  
0        21C  R1       8          1917150  
1        21C  R1      16          2168035  
2        21C  R1      24          2068099  
3        21C  R1      40          2118452  
4        21C  R2       8          2111700  
..       ...  ..     ...              ...  
63     pH7.3  R1      40          2719186  
64     pH7.3  R2       8          2757245  
65     pH7.3  R2      16          2680456  
66     pH7.3  R2      24          2585981  
67     pH7.3  R2      40          2769573  

[68 rows x 8 columns]

----------
#### Filter samples by total counts

In [15]:
# Create a column in samplesInfo_hBFA1 to indicate which samples are excluded from analysis.
samplesInfo_hBFA1['exclude_from_analysis'] = False

In [16]:
MIN_SAMPLE_COUNT_TOTAL = 1e5

In [17]:
# Exclude treatments that have one or more samples with total count below threshold
good_coverage_samples = []
for treatment in samplesInfo_hBFA1['treatment'].unique():
    treatmentInfo = samplesInfo_hBFA1[samplesInfo_hBFA1['treatment'] == treatment]
    if(np.all((treatmentInfo['timept'] == 0) | (treatmentInfo['total_raw_count'] >= MIN_SAMPLE_COUNT_TOTAL))):
        good_coverage_samples.extend( treatmentInfo['sample'].values )
    else:
        samplesInfo_hBFA1.loc[samplesInfo_hBFA1['treatment'] == treatment, 'exclude_from_analysis'] = True

In [18]:
rawCounts_hBFA1   = rawCounts_hBFA1[good_coverage_samples]

In [19]:
# Drop excluded samples from the data set
samplesInfo_hBFA1 = samplesInfo_hBFA1[~samplesInfo_hBFA1['exclude_from_analysis']]
samplesInfo_hBFA1 = samplesInfo_hBFA1.drop(columns=['exclude_from_analysis'])

------

#### Get barcode sequence along with GC-content for each variant

In [20]:
variants_GCratios = []
for idx, variant_info in variantsInfo_hBFA1.iterrows():
    variant_barcodeSeq = variant_info['barcode_full']
    variant_barcodeGCratio = (variant_barcodeSeq.count('G') + variant_barcodeSeq.count('C'))/len(variant_barcodeSeq)
    variants_GCratios.append(variant_barcodeGCratio)
#----------------
variantsInfo_hBFA1['barcode_GCratio']  = variants_GCratios
variantsInfo_hBFA1

barcode_full  \
0     ATAAAAAAGCACAAGCCTTTTGACGACACCTAAATTAGTTATCCAT...   
1     CGATAAACCCACAATATGATTATCGGTACACAAGAGGGTTTCTGTT...   
2     CGCACAAGAAAGAATAATCTTGAATTGGTAAAACCAGATTTGGCAT...   
3     ACAAAAAGATATAACAAGCTTGAAGACCCGAAAAAGTTTTTTATCT...   
4     AATAAAAGAAGGAAAAGCATTTAAACAAACAAACTTTCTTTTTTCT...   
...                                                 ...   
2581  CACGCAAAACTTTATTCAATAACTTCGTCCGCCAATCATATTGTAC...   
2582  CTATGAAAAGACAACAAGATTGCCTACTATTAACTGATTTTTCTCT...   
2583  CGTTAGAATTCTAAACCTGGTTCAACTATTGGAACATCTTTAGTAT...   
2584  CCGTCAATCATTAACTTAGTTTCTTACCAGAAATCATATTTTTATT...   
2585  TTATTAATATAGAAAGCAATTTTAATAGAAAAACCATTTTCGTATT...   

                   barcode_diverse         barcode_environment  total_counts  \
0       ATAAAAAAGCACAAGCCTTTTGACGA  CACCTAAATTAGTTATCCATTCGGCT       9616313   
1       CGATAAACCCACAATATGATTATCGG  TACACAAGAGGGTTTCTGTTTTATGG       6648391   
2       CGCACAAGAAAGAATAATCTTGAATT  GGTAAAACCAGATTTGGCATTCACTA       3368497   
3       ACAAAAAGATATAACAAGCTTGAAGA  CCCGAAAAAGTTTTTTATCTTCAATG       2082320   
4       AATAAAAGAAGGAAAAGCATTTAAAC  AAACAAACTTTCTTTTTTCTTTATGC       2013526   
...                            ...                         ...           ...   
2581  CACGCAAAACTTTATTCAATAACTTCGT  CCGCCAATCATATTGTACTTTCAAGG           267   
2582    CTATGAAAAGACAACAAGATTGCCTA  CTATTAACTGATTTTTCTCTTTCGAA           243   
2583   CGTTAGAATTCTAAACCTGGTTCAACT  ATTGGAACATCTTTAGTATTTAATAC           214   
2584    CCGTCAATCATTAACTTAGTTTCTTA  CCAGAAATCATATTTTTATTTATTAA           139   
2585    TTATTAATATAGAAAGCAATTTTAAT  AGAAAAACCATTTTCGTATTTTATAG            96   

     subpool_environment                which_subpools putative_environment  \
0               not_read                      not_read                SC_2N   
1               not_read                      not_read             SC_alpha   
2          GlyEtOH_alpha  -R1-Subpool.-R1-autodiploids        GlyEtOH_alpha   
3             FLC4_alpha                   -R1-Subpool           FLC4_alpha   
4              YPD_alpha                   -R1-Subpool      contam-pH3_8_2N   
...                  ...                           ...                  ...   
2581            not_read                      not_read             SC_alpha   
2582         pH7_3_alpha                   -R2-Subpool   sanger-pH7_3_alpha   
2583           37C_alpha                   -R1-Subpool              unknown   
2584           21C_alpha                   -R1-Subpool            21C_alpha   
2585           YPD_alpha                   -R1-Subpool            YPD_alpha   

      barcode_GCratio  
0            0.365385  
1            0.384615  
2            0.346154  
3            0.288462  
4            0.230769  
...               ...  
2581         0.370370  
2582         0.307692  
2583         0.301887  
2584         0.230769  
2585         0.173077  

[2586 rows x 8 columns]

------

#### Label control set variants ("putatively neutral barcodes")

In [21]:
variantsInfo_hBFA1['neutral_group'] = ''
variantsInfo_hBFA1.loc[variantsInfo_hBFA1['putative_environment'] == 'YPD_alpha', 'neutral_group'] = 'YPD_alpha'

variantsInfo_hBFA1['control_set'] = False
variantsInfo_hBFA1.loc[variantsInfo_hBFA1['putative_environment'] == 'YPD_alpha', 'control_set'] = True

In [22]:
print(f"{np.sum(variantsInfo_hBFA1['control_set'].values)}/{len(variantsInfo_hBFA1)}")

302/2586


#### Save processed data to file

In [23]:
samplesInfo_hBFA1.to_csv(outdir+'samples-hBFA1.csv', index=False)
samplesInfo_hBFA1

chen_sample_label             sample           assay assay_set  \
0      hBFA1-21C-R1-Time8     hBFA1:21C-R1-8    hBFA1:21C-R1     hBFA1   
1     hBFA1-21C-R1-Time16    hBFA1:21C-R1-16    hBFA1:21C-R1     hBFA1   
2     hBFA1-21C-R1-Time24    hBFA1:21C-R1-24    hBFA1:21C-R1     hBFA1   
3     hBFA1-21C-R1-Time40    hBFA1:21C-R1-40    hBFA1:21C-R1     hBFA1   
4      hBFA1-21C-R2-Time8     hBFA1:21C-R2-8    hBFA1:21C-R2     hBFA1   
..                    ...                ...             ...       ...   
63  hBFA1-pH7_3-R1-Time40  hBFA1:pH7.3-R1-40  hBFA1:pH7.3-R1     hBFA1   
64   hBFA1-pH7_3-R2-Time8   hBFA1:pH7.3-R2-8  hBFA1:pH7.3-R2     hBFA1   
65  hBFA1-pH7_3-R2-Time16  hBFA1:pH7.3-R2-16  hBFA1:pH7.3-R2     hBFA1   
66  hBFA1-pH7_3-R2-Time24  hBFA1:pH7.3-R2-24  hBFA1:pH7.3-R2     hBFA1   
67  hBFA1-pH7_3-R2-Time40  hBFA1:pH7.3-R2-40  hBFA1:pH7.3-R2     hBFA1   

   treatment rep  timept  total_raw_count  
0        21C  R1       8          1917150  
1        21C  R1      16          2168035  
2        21C  R1      24          2068099  
3        21C  R1      40          2118452  
4        21C  R2       8          2111700  
..       ...  ..     ...              ...  
63     pH7.3  R1      40          2719186  
64     pH7.3  R2       8          2757245  
65     pH7.3  R2      16          2680456  
66     pH7.3  R2      24          2585981  
67     pH7.3  R2      40          2769573  

[68 rows x 8 columns]

In [24]:
variantsInfo_hBFA1.to_csv(outdir+'variants-hBFA1_withNeutralGroups.csv', index=False)
variantsInfo_hBFA1

barcode_full  \
0     ATAAAAAAGCACAAGCCTTTTGACGACACCTAAATTAGTTATCCAT...   
1     CGATAAACCCACAATATGATTATCGGTACACAAGAGGGTTTCTGTT...   
2     CGCACAAGAAAGAATAATCTTGAATTGGTAAAACCAGATTTGGCAT...   
3     ACAAAAAGATATAACAAGCTTGAAGACCCGAAAAAGTTTTTTATCT...   
4     AATAAAAGAAGGAAAAGCATTTAAACAAACAAACTTTCTTTTTTCT...   
...                                                 ...   
2581  CACGCAAAACTTTATTCAATAACTTCGTCCGCCAATCATATTGTAC...   
2582  CTATGAAAAGACAACAAGATTGCCTACTATTAACTGATTTTTCTCT...   
2583  CGTTAGAATTCTAAACCTGGTTCAACTATTGGAACATCTTTAGTAT...   
2584  CCGTCAATCATTAACTTAGTTTCTTACCAGAAATCATATTTTTATT...   
2585  TTATTAATATAGAAAGCAATTTTAATAGAAAAACCATTTTCGTATT...   

                   barcode_diverse         barcode_environment  total_counts  \
0       ATAAAAAAGCACAAGCCTTTTGACGA  CACCTAAATTAGTTATCCATTCGGCT       9616313   
1       CGATAAACCCACAATATGATTATCGG  TACACAAGAGGGTTTCTGTTTTATGG       6648391   
2       CGCACAAGAAAGAATAATCTTGAATT  GGTAAAACCAGATTTGGCATTCACTA       3368497   
3       ACAAAAAGATATAACAAGCTTGAAGA  CCCGAAAAAGTTTTTTATCTTCAATG       2082320   
4       AATAAAAGAAGGAAAAGCATTTAAAC  AAACAAACTTTCTTTTTTCTTTATGC       2013526   
...                            ...                         ...           ...   
2581  CACGCAAAACTTTATTCAATAACTTCGT  CCGCCAATCATATTGTACTTTCAAGG           267   
2582    CTATGAAAAGACAACAAGATTGCCTA  CTATTAACTGATTTTTCTCTTTCGAA           243   
2583   CGTTAGAATTCTAAACCTGGTTCAACT  ATTGGAACATCTTTAGTATTTAATAC           214   
2584    CCGTCAATCATTAACTTAGTTTCTTA  CCAGAAATCATATTTTTATTTATTAA           139   
2585    TTATTAATATAGAAAGCAATTTTAAT  AGAAAAACCATTTTCGTATTTTATAG            96   

     subpool_environment                which_subpools putative_environment  \
0               not_read                      not_read                SC_2N   
1               not_read                      not_read             SC_alpha   
2          GlyEtOH_alpha  -R1-Subpool.-R1-autodiploids        GlyEtOH_alpha   
3             FLC4_alpha                   -R1-Subpool           FLC4_alpha   
4              YPD_alpha                   -R1-Subpool      contam-pH3_8_2N   
...                  ...                           ...                  ...   
2581            not_read                      not_read             SC_alpha   
2582         pH7_3_alpha                   -R2-Subpool   sanger-pH7_3_alpha   
2583           37C_alpha                   -R1-Subpool              unknown   
2584           21C_alpha                   -R1-Subpool            21C_alpha   
2585           YPD_alpha                   -R1-Subpool            YPD_alpha   

      barcode_GCratio neutral_group  control_set  
0            0.365385                      False  
1            0.384615                      False  
2            0.346154                      False  
3            0.288462                      False  
4            0.230769                      False  
...               ...           ...          ...  
2581         0.370370                      False  
2582         0.307692                      False  
2583         0.301887                      False  
2584         0.230769                      False  
2585         0.173077     YPD_alpha         True  

[2586 rows x 10 columns]

In [25]:
rawCounts_hBFA1.to_csv(outdir+'counts-hBFA1.csv', index=False)
rawCounts_hBFA1

hBFA1:21C-R1-8  hBFA1:21C-R1-16  hBFA1:21C-R1-24  hBFA1:21C-R1-40  \
0              18075            34976            73925           379540   
1             109316           117037           102934            81035   
2               6539             4508             4751             3545   
3              31099            42058            42356            38478   
4               4250             7508            13340            61474   
...              ...              ...              ...              ...   
2581               1                2                4                2   
2582               3                3                5                1   
2583               3                5                4                2   
2584               5                4                4                2   
2585               4                4                3                2   

      hBFA1:21C-R2-8  hBFA1:21C-R2-16  hBFA1:21C-R2-24  hBFA1:21C-R2-40  \
0              19395            33224            67414           473720   
1             122720           113384            95658            83279   
2               7334             4549             4273             3511   
3              39228            41195            42935            51296   
4               4878             7708            13778            94633   
...              ...              ...              ...              ...   
2581               5                6                7                9   
2582               7                5                5                5   
2583               3                7                6                3   
2584               8                6                2               13   
2585               2                3                2                6   

      hBFA1:37C-R1-8  hBFA1:37C-R1-16  hBFA1:37C-R1-24  hBFA1:37C-R1-40  \
0              36413            72215           120048           323260   
1             140343           131053           103226            65894   
2               7771             6269             9488            11685   
3              32634            23369            18619             7488   
4               6576            14705            29220            76955   
...              ...              ...              ...              ...   
2581               7                2                3                2   
2582               4                0                4                3   
2583               1                0                3                3   
2584               0                0                4                0   
2585               0                0                5                1   

      hBFA1:37C-R2-8  hBFA1:37C-R2-16  hBFA1:37C-R2-24  hBFA1:37C-R2-40  \
0              35264            76376           128764           395642   
1             161542           152263           127606            76375   
2              12360             7577             5588            13405   
3              40855            28633            17674             7408   
4               7438            15378            20278            80289   
...              ...              ...              ...              ...   
2581               4                7                3                6   
2582               2                3                5                4   
2583               6                4                3                1   
2584               1                0                0                0   
2585               1                3                0                1   

      hBFA1:48Hr-R1-8  hBFA1:48Hr-R1-16  hBFA1:48Hr-R1-24  hBFA1:48Hr-R1-40  \
0               58079            106586            153886            503009   
1               81672             92562             84147             92839   
2                5796              4743             18323             22243   
3               34917             56809             34105             67315   
4      